<a href="https://colab.research.google.com/github/devansurf/Carlini-WagnerL2/blob/main/Carlini%26Wagner_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carlini & Wagner (L2) Adversarial Attack
Load pretrained models from HuggingFace, run attacks and plot images and any relevant metrics

In [ ]:
%pip install torchattacks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 13.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.wh

In [ ]:
%pip install accelerate -U

IMPORTS!!!

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch
import torch.optim as optim
import torchattacks
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xml.etree.ElementTree as ET
import os

from transformers import TrainingArguments, Trainer, default_data_collator
from transformers import ViTImageProcessor, BeitImageProcessor, ConvNextImageProcessor
from transformers import ViTForImageClassification, AutoModelForImageClassification
from transformers import ViTFeatureExtractor, BeitFeatureExtractor, ConvNextFeatureExtractor

from PIL import Image
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.metrics import classification_report, f1_score, confusion_matrix

from torch import nn


In [ ]:
# CPU or GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

using_kaggle = False
using_torch = False

In [ ]:
#Confusion matrix
def generate_confusion_matrix(actual_labels, predicted_labels, classes):
    cm = confusion_matrix(actual_labels, predicted_labels)


    fig = plt.figure(figsize=(16, 14))
    ax = plt.subplot()
    sns.heatmap(cm, annot=True, ax=ax, fmt="g")
    ax.set_xlabel("Predicted", fontsize=20)
    ax.xaxis.set_label_position("bottom")
    plt.xticks(rotation=90)
    ax.xaxis.tick_bottom()

    ax.set_ylabel("True", fontsize=20)
    plt.yticks(rotation=0)

    plt.title("Confusion Matrix", fontsize=20)

    plt.show()

    F1_score = f1_score(actual_labels, predicted_labels, average='macro')

    print(f'f1_score: {F1_score}')

In [ ]:
def plot_images(x, y, class_names, x_adv=None, y_adv=None):
    if x_adv is not None:
        fig, ax = plt.subplots(3, 10, figsize=(20, 5))
    else:
        fig, ax = plt.subplots(1, 10, figsize=(20, 5))

    for i in range(10):
        if x_adv is not None:
            ax[0, i].imshow(x[i].permute(1, 2, 0))
            ax[0, i].set_title(class_names[y[i]])
            ax[0, i].axis("off")
            ax[1, i].imshow(x_adv[i].detach().cpu().permute(1, 2, 0))
            if y_adv[i].item() != y[i].item():
                ax[1, i].set_title(class_names[y_adv[i]], color="r")
            else:
                ax[1, i].set_title(class_names[y_adv[i]], color="k")
            ax[1, i].axis("off")
            ax[2, i].imshow(
                (torch.abs(x[i] - x_adv[i].detach().cpu()) * 25)
                .clamp_(0, 1)
                .permute(1, 2, 0)
            )
            ax[2, i].axis("off")
        else:
            ax[i].imshow(x[i].permute(1, 2, 0))
            ax[i].set_title(class_names[y[i]])
            ax[i].axis("off")

In [ ]:
class HuggingFaceModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingFaceModelWrapper, self).__init__()
        # Load the pretrained model

        self.model = model
        #Hardcoded output layer (That of CIFAR10 num_classes)
        #self.model.classifier = nn.Linear(self.model.classifier.in_features, 10)

    def forward(self, pixel_values):
        if not isinstance(pixel_values, torch.Tensor):
            raise ValueError("pixel_values must be a torch.Tensor")

        # Hugging Face models expect a dictionary with 'pixel_values' as the key
        outputs = self.model(pixel_values=pixel_values)

        # Extract logits
        logits = outputs.logits

        return logits

HuggingFace pretrained models

In [ ]:
# ViT model
model_name_vit = "google/vit-base-patch16-224-in21k"
feature_extractor_vit = ViTFeatureExtractor.from_pretrained(model_name_vit)
model_vit = ViTForImageClassification.from_pretrained(model_name_vit, num_labels=10)

# BEiT model
# model_name_beit = "microsoft/beit-base-patch16-224-pt22k-ft22k"
# feature_extractor_beit = BeitFeatureExtractor.from_pretrained(model_name_beit)
# model_beit = AutoModelForImageClassification.from_pretrained(model_name_beit)

# ConvNeXt model
# model_name_convnext = "facebook/convnext-base-224"
# feature_extractor_convnext = ConvNextFeatureExtractor.from_pretrained(model_name_convnext)
# model_convnext = AutoModelForImageClassification.from_pretrained(model_name_convnext, ignore_mismatched_sizes = True, num_labels=10)

# model_vgg = models.vgg16(pretrained=True)
model_vgg.classifier[6] = torch.nn.Linear(4096, 10)  # Adjusting the last layer for 10 classes

# if using_kaggle:
#     model_convnext = AutoModelForImageClassification.from_pretrained(model_name_convnext, ignore_mismatched_sizes = True, num_labels=4)


HuggingFace Wrapper function to handle outputs for torchattacks

In [ ]:
if using_kaggle:
    !pip install kaggle

In [ ]:
# We create a hidden directory to store our kaggle API key that would let us run
# everything else
from google.colab import files

if using_kaggle:
    files.upload()

    !mkdir -p ~/.kaggle

    !cp kaggle.json ~/.kaggle/

    !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
if using_kaggle:
    !kaggle datasets download -d andrewmvd/road-sign-detection


In [ ]:
if using_kaggle:
    !unzip road-sign-detection.zip

#Gather images and annotations from Kaggle dataset


In [ ]:
#Dataset focused on roadsigns
class KaggleHuggingFace(Dataset):
    def __init__(self, img_dir, annotations_dir, split='train'):
        """
        Args:
            img_dir (string): Path to the images directory.
            annotations_dir (string): Path to the annotations directory.
            split (string, optional): Indicates if the dataset is for training or testing. Defaults to 'train'.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.img_dir = img_dir
        self.annotations_dir = annotations_dir

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)), #model input size
            transforms.ToTensor(),
        ])

        self.split = split
        self.images = os.listdir(img_dir)
        self.labels = self._extract_labels()

        # Split dataset into train/test if necessary
        if self.split == 'train':
            self.images, self.labels = self._split_dataset(self.images, self.labels, train=True)
        else:
            self.images, self.labels = self._split_dataset(self.images, self.labels, train=False)

        self.classes = sorted(list(set(self.labels.values())))

    def _extract_labels(self):
        labels = {}
        for annotation in os.listdir(self.annotations_dir):
            xml_path = os.path.join(self.annotations_dir, annotation)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            filename = root.find('filename').text
            object_name = root.find('.//object/name').text  # Assumes one main object per image
            labels[filename] = object_name
        return labels

    def _split_dataset(self, images, labels, train=True, train_size=0.8):
        # Split images based on split flag
        split_idx = int(len(images) * train_size)
        if train:
            selected_images = images[:split_idx]
        else:
            selected_images = images[split_idx:]

        # Filter labels to match selected images
        selected_labels = {k: v for k, v in labels.items() if k in selected_images}
        return selected_images, selected_labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.labels[img_name]
        label_idx = self.classes.index(label)

        if self.transform:
            image = self.transform(image)
        if self.split == 'test':
            return image, label_idx
        return {'pixel_values': image, 'labels': label_idx}

In [ ]:
class CIFAR10HuggingFace(Dataset):
    def __init__(self, split='train', subset_size=1000):
        self.split = split
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)), #model input size
            transforms.ToTensor(),
        ])

         # Load the full dataset
        self.dataset = datasets.CIFAR10(root='./data', train=(split == 'train'),
                                        download=True, transform=self.transform)
        self.classes = self.dataset.classes

        if (split == 'train'):
            # Generate a random subset of indices
            if subset_size > len(self.dataset):
                raise ValueError("Subset size is larger than the dataset size.")
            subset_indices = np.random.choice(len(self.dataset), subset_size, replace=False)

            # Create a dataset subset
            self.dataset = Subset(self.dataset, subset_indices)



    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return {'pixel_values': image, 'labels': label}


In [ ]:
class TorchModelWrapper(nn.Module):
    def __init__(self, model, num_classes=10):
        super(TorchModelWrapper, self).__init__()
        self.model = model
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)

    def forward(self, inputs):
        pixel_values = inputs['pixel_values']
        outputs = self.model(pixel_values)
        return {'logits': outputs}

In [ ]:
#Fine-Tune model
train_dataset = CIFAR10HuggingFace(split='train', subset_size=10000)
eval_dataset = CIFAR10HuggingFace(split='test')
test_dataset = datasets.CIFAR10(root='./data', train=False,
            download=True,
            transform=transforms.Compose([
            transforms.Resize((224, 224)), #model input size
            transforms.ToTensor(),
        ])
)

if using_kaggle:
    train_dataset = KaggleHuggingFace('/content/images', '/content/annotations', split='train')
    eval_dataset = KaggleHuggingFace('/content/images', '/content/annotations', split='validation')
    test_dataset = KaggleHuggingFace('/content/images', '/content/annotations', split='test')

def train_model(model, feature_extractor = None):
    model = model.to(device)
    training_args = TrainingArguments(
        output_dir='./results',          # Where to store the model checkpoints
        num_train_epochs=9,              # Number of training epochs
        per_device_train_batch_size=16,   # Training batch size
        per_device_eval_batch_size=16,    # Evaluation batch size
        warmup_steps=500,                # Number of warmup steps
        weight_decay=0.01,               # Weight decay
        logging_dir='./logs',            # Where to store logs
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=default_data_collator, #not usually used in Image Processing, but used mostly in NLP's
        tokenizer=feature_extractor,
    )
    trainer.train()
    trainer.evaluate()

In [ ]:
def torch_train(model, learning_rate = 0.001, epochs = 10, batch_size = 32):

    train_dataset = datasets.CIFAR10(root='./data', train=True,
            download=True,
            transform=transforms.Compose([
            transforms.Resize((224, 224)), #model input size
            transforms.ToTensor(),
        ]))

    model = model.to(device)

    # Optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

In [ ]:
#Train all models

train_model(model_vit, feature_extractor_vit)
model_vit = HuggingFaceModelWrapper(model_vit)

# train_model(model_beit, feature_extractor_beit)
# model_beit = HuggingFaceModelWrapper(model_beit)

# train_model(model_convnext, feature_extractor_convnext)
# model_convnext = HuggingFaceModelWrapper(model_convnext)
# torch_train(model_vgg)


Lets start making some predictions

In [ ]:
def run_attack(model):
    #atk = torchattacks.CW(model.to(device), c=1, steps=1000, lr=0.01)
    atk = torchattacks.OnePixel(model, pixels=10, steps=10, popsize=10, inf_batch=128)
    dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    for image, label in dataloader:
        image, label = image.to(device), label.to(device)

        #make prediction without using adversarial images
        with torch.no_grad():
            predictions = model(image)
        predicted_label = torch.argmax(predictions, dim=1)

        plot_images(image.cpu(), predicted_label.cpu(), test_dataset.classes)
        generate_confusion_matrix(predicted_label.cpu(), label.cpu(), test_dataset.classes)

        # Create adversarial images
        adv_image = atk(image,label)

        # Make predictions using adversarial images
        with torch.no_grad():
            predictions = model(adv_image)
        adv_label = torch.argmax(predictions, dim=1)


        #plot the results
        plot_images(image.cpu(), label.cpu(), test_dataset.classes, x_adv=adv_image.cpu(), y_adv=adv_label.cpu())
        generate_confusion_matrix(adv_label.cpu(), label.cpu(), test_dataset.classes)

        break

In [ ]:
run_attack(model_vit)
#run_attack(model_beit)
#run_attack(model_convnext)
#run_attack(model_vgg, learning_rate = 0.001, epochs = 10, batch_size = 32)

Function that helps plot images

Lets use our previously defined function (taken from MITLL) to plot our adversarial images